In [32]:
import os

In [33]:
os.chdir(r"C:\Users\shash\OneDrive\Desktop\Summer Project\End-to-End-Chest-Cancer-Classification-using-MLFlow-and-DVC")


In [34]:
%pwd

'C:\\Users\\shash\\OneDrive\\Desktop\\Summer Project\\End-to-End-Chest-Cancer-Classification-using-MLFlow-and-DVC'

In [35]:
from dotenv import load_dotenv
load_dotenv()



True

In [36]:

import tensorflow as tf

In [37]:

model = tf.keras.models.load_model("artifacts/training/model.h5")


In [38]:

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [43]:
from pathlib import Path
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf


CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts\data_ingestion\Chest-Ct-scan-data",
            mlflow_uri="https://dagshub.com/choubeyshashwat2/End-to-End-Chest-Cancer-Classification-using-MLFlow-and-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE


        )
        return eval_config


In [44]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [45]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score
    
    def save_score(self):
        scores = {"loss": self.score[0],"accuracy": self.score[1]}
        save_json(path=Path("scores.json"),data=scores)
    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy":self.score[1]}

            )

            if tracking_url_type_store != "file":
                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

    


In [46]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e


[2025-05-17 16:10:24,608: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-17 16:10:24,611: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-17 16:10:24,612: INFO: common: created directory at: artifacts]


Found 139 images belonging to 2 classes.
9/9 [==============================] - 11s 1s/step - loss: 0.0396 - accuracy: 0.9856


2025/05/17 16:10:40 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-05-17 16:10:43,564: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\shash\AppData\Local\Temp\tmpr550_e1k\model\data\model\assets
[2025-05-17 16:10:44,147: INFO: builder_impl: Assets written to: C:\Users\shash\AppData\Local\Temp\tmpr550_e1k\model\data\model\assets]


c:\Users\shash\.conda\envs\cancer\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Successfully registered model 'VGG16Model'.
2025/05/17 16:12:30 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
